# RISS에서 학위 논문 데이터 수집하기

In [1]:
import requests
import math
import pickle
import time
import sys
import pandas as pd
from tqdm.notebook import tqdm
from random import uniform
import lxml
import lxml.etree as et
from bs4 import BeautifulSoup
import glob
import urllib3
urllib3.disable_warnings()
from natsort import natsorted
import re
from concurrent.futures import ThreadPoolExecutor
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "DNT": "1", # Do Not Track 요청 헤더 (사용자 추적 거부)
    # 필요한 추가 헤더를 여기에 추가
}

In [3]:
def request_until_success(url, headers, timeout=7, delay=3, max_retries=5, backoff_factor=2):
    """
    Continuously makes a request to the specified URL with a timeout until the request is successful
    or the maximum number of retries is reached.

    Args:
    - url (str): The URL to request.
    - headers (dict): The headers to include in the request.
    - timeout (int): The timeout for the request in seconds.
    - delay (int): The initial delay between retries in seconds.
    - max_retries (int): The maximum number of retries before giving up.
    - backoff_factor (int): The factor by which to multiply the delay after each retry.

    Returns:
    - response: The successful response from the server.

    Raises:
    - requests.RequestException: If the request fails after max_retries attempts.
    """
    attempt = 1
    while attempt <= max_retries:
        try:
            response = requests.get(url, headers=headers, timeout=timeout)
            response.raise_for_status()  # Raises HTTPError for bad responses
            return response
        except requests.Timeout as e:
            print(f"Request timed out (Attempt {attempt}): {e}")
        except requests.RequestException as e:
            print(f"Request failed (Attempt {attempt}): {e}")

        if attempt == max_retries:
            raise requests.RequestException(f"Max retries reached: Failed to get a successful response from {url}")

        # 지수적 백오프 적용
        sleep_time = delay * (backoff_factor ** (attempt - 1))
        print(f"Retrying in {sleep_time} seconds...")
        time.sleep(sleep_time)
        attempt += 1

### 검색 조건에 맞는 URL 설정
학과정보 : 국어국문 OR 국문  
학위수여연도 : 2020~2024    
학위유형 : 국내박사  
페이지당 출력 : 100개씩  
https://www.riss.kr/search/Search.do?isDetailSearch=Y&searchGubun=true&viewYn=OP&queryText=znMajor%2C%EA%B5%AD%EC%96%B4%EA%B5%AD%EB%AC%B8%40op%2COR%40znMajor%2C%EA%B5%AD%EB%AC%B8&strQuery=&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=2020&p_year2=2024&iStartCount=0&orderBy=&mat_type=&mat_subtype=T2&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&db_type=&image_yn=&gubun=&kdc=&ttsUseYn=&l_sub_code=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=bib_t&colName=bib_t&pageScale=100&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=

In [28]:
# 검색 조건을 넣은 url을 아래에 복사해 넣으세요
# url = 'https://www.riss.kr/search/Search.do?isDetailSearch=Y&searchGubun=true&viewYn=OP&queryText=znMajor%2C%EA%B5%AD%EC%96%B4%EA%B5%AD%EB%AC%B8%40op%2COR%40znMajor%2C%EA%B5%AD%EB%AC%B8&strQuery=&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=2020&p_year2=2024&iStartCount=0&orderBy=&mat_type=&mat_subtype=T2&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&db_type=&image_yn=&gubun=&kdc=&ttsUseYn=&l_sub_code=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=bib_t&colName=bib_t&pageScale=100&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query='
url = 'https://www.riss.kr/search/Search.do?isDetailSearch=Y&searchGubun=true&viewYn=OP&queryText=znMajor%2C%EA%B5%AD%EC%96%B4%EA%B5%AD%EB%AC%B8%40op%2COR%40znMajor%2C%EA%B5%AD%EB%AC%B8&strQuery=&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=2000&p_year2=2024&iStartCount=100&orderBy=&mat_type=&mat_subtype=T1&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&db_type=&image_yn=&gubun=&kdc=&ttsUseYn=&l_sub_code=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=bib_t&colName=bib_t&pageScale=100&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query='

In [29]:
req = request_until_success(url,headers).text

In [30]:
soup = BeautifulSoup(req, 'lxml')

In [31]:
cont_ml60_classes = soup.find_all('div', class_='cont ml60')

data = [{
    'title': cont_ml60.find('p', class_='title').get_text(strip=True) if cont_ml60.find('p', class_='title') else None,
    'link': cont_ml60.find('p', class_='title').find('a').get('href') if cont_ml60.find('p', class_='title') and cont_ml60.find('p', class_='title').find('a') else None,
    'writer': cont_ml60.find('p', class_='etc').find_all('span')[0].get_text(strip=True) if cont_ml60.find('p', class_='etc') and len(cont_ml60.find('p', class_='etc').find_all('span')) > 0 else None,
    'assigned': cont_ml60.find('p', class_='etc').find_all('span')[1].get_text(strip=True) if cont_ml60.find('p', class_='etc') and len(cont_ml60.find('p', class_='etc').find_all('span')) > 1 else None,
    'year': cont_ml60.find('p', class_='etc').find_all('span')[2].get_text(strip=True) if cont_ml60.find('p', class_='etc') and len(cont_ml60.find('p', class_='etc').find_all('span')) > 2 else None,
    'grad': cont_ml60.find('p', class_='etc').find_all('span')[3].get_text(strip=True) if cont_ml60.find('p', class_='etc') and len(cont_ml60.find('p', class_='etc').find_all('span')) > 3 else None,
    'preAbstract': cont_ml60.find('p', class_='preAbstract').get_text(strip=True) if cont_ml60.find('p', class_='preAbstract') else None
} for cont_ml60 in cont_ml60_classes]

In [32]:
data

[{'title': '한국어와 중국어의 3인칭 대명사 대조연구',
  'link': '/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=c0721d4cd14735acffe0bdc3ef48d419&keyword=',
  'writer': '웨이팅',
  'assigned': '서울시립대학교',
  'year': '2015',
  'grad': '국내석사',
  'preAbstract': "本論文以韓語和漢語的第三人稱代詞爲中心,通過語料庫分析第三人稱分別在書面語和口語中的使用頻率,利用韓中小說,童話,電視劇和電影中的實際使用例句進行了比較研究,找到了兩者間的共性和差異。\n  第一章介紹了本論文的研究目的,研究對象和研究方法。本論文旨在通過對比韓語和漢語的第三人稱代詞,發掘出二者的共同點和差異點,闡明它們在書面語和口語中的實際使用方法,爲學習韓語的漢語圈實習者提供一個可參照的樣本,幫助他們更好的聽說讀寫韓語。第三人稱代詞在文章寫作和實際對話中扮演著重要的角色,它能使對話變得簡潔,指示變得明確。本章還考察了現有的先行研究,找出了現存研究中的不足之處。即,當前關于韓中人稱代詞的對比研究已經有了一些成果,但是沒有專門研究對照韓中第三人稱的論文,特別是以口語爲中心的對比研究幾乎處于尚未開始的階段。本論文以韓國國立國語研究院出版的《主要詞彙用例集:副詞ㆍ冠形詞ㆍ代名詞ㆍ數詞ㆍ感歎詞ㆍ依存名詞》中收錄的第三人稱代詞,漢語以“他(們),她(們)”作爲主要研究對象。\n  第二章中介紹了韓語和漢語中第三人稱代詞的定義和概念。韓語和漢語中的第三人稱代詞都是指代說者和聽者以外的第三方的。\n  第三章簡略考察韓語和漢語的第三人代詞的曆史發展過程。本論文以前人研究爲基礎,將韓語劃分爲古代國語,中世國語,近代國語和現代國語四個時期進行考察;將漢語劃分爲上古漢語,中古漢語,近代漢語和現代漢語四個時期進行考察。古代國語時期用做第三人稱代詞的韓語有“此,伊,彼”;中世國語時期用做第三人稱代詞的韓語有“是,斯,之,然”;20世紀初受西方文學的影響,近代國語仿造日語創造了女性第三人稱代詞“그녀”;現代國語中用做第三人稱代詞的韓語以“이,그,저”和“이,그,저 + 의존명사/명사

### 기본 정보 자동으로 수집하기

In [33]:
# 총 논문 개수 확인(total_count)
# Assuming 'soup' is your BeautifulSoup object
num_span = soup.find('span', class_='num')

if num_span is not None:
    total_count = int(num_span.get_text().replace(',', ''))
else:
    total_count = 0

total_count

12450

In [34]:
# 수집용 URL 세팅
# The base URL without the 'iStartCount' parameter
# Parse the URL
parsed_url = urlparse(url)

# Parse the query parameters
params = parse_qs(parsed_url.query)

# Remove the 'iStartCount' parameter
params.pop('iStartCount', None)

# Re-encode the query parameters
new_query = urlencode(params, doseq=True)

# Replace the old query parameters with the new ones
new_url = parsed_url._replace(query=new_query)

# Unparse the URL
base_url = urlunparse(new_url)

print(base_url)

https://www.riss.kr/search/Search.do?isDetailSearch=Y&searchGubun=true&viewYn=OP&queryText=znMajor%2C%EA%B5%AD%EC%96%B4%EA%B5%AD%EB%AC%B8%40op%2COR%40znMajor%2C%EA%B5%AD%EB%AC%B8&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=2000&p_year2=2024&mat_subtype=T1&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&icate=bib_t&colName=bib_t&pageScale=100&isTab=Y


##### 실제 수집 코드

In [ ]:
# The base URL without the 'iStartCount' parameter
# base_url = "https://www.riss.kr/search/Search.do?isDetailSearch=Y&searchGubun=true&viewYn=OP&queryText=znMajor,국어국문@op,OR@znMajor,국문&strQuery=&exQuery=&exQueryText=&order=/DESC&onHanja=false&strSort=RANK&p_year1=2000&p_year2=2023&orderBy=&mat_type=&mat_subtype=T2&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&db_type=&image_yn=&gubun=&kdc=&ttsUseYn=&l_sub_code=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=bib_t&colName=bib_t&pageScale=100&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query="

# Calculate the number of iterations
iterations = math.ceil(total_count / 100)

# Initialize an empty list to store the data
data = []

# 멀티 쓰레딩 활용 데이터 수집 함수
def fetch_data(i):
    url = f"{base_url}&iStartCount={i * 100}"
    response = request_until_success(url, headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    cont_ml60_classes = soup.find_all('div', class_='cont ml60')

    # 필요한 데이터만 한 번에 추출
    data = []
    for cont_ml60 in cont_ml60_classes:
        title = cont_ml60.find('p', class_='title')
        etc = cont_ml60.find('p', class_='etc')
        spans = etc.find_all('span') if etc else []

        data.append({
            'title': title.get_text(strip=True) if title else None,
            'link': title.find('a').get('href') if title and title.find('a') else None,
            'writer': spans[0].get_text(strip=True) if len(spans) > 0 else None,
            'assigned': spans[1].get_text(strip=True) if len(spans) > 1 else None,
            'year': spans[2].get_text(strip=True) if len(spans) > 2 else None,
            'grad': spans[3].get_text(strip=True) if len(spans) > 3 else None,
            'preAbstract': cont_ml60.find('p', class_='preAbstract').get_text(strip=True) if cont_ml60.find('p', class_='preAbstract') else None
        })
    return data

# ThreadPoolExecutor를 사용하여 멀티스레딩 구현
with ThreadPoolExecutor(max_workers=100) as executor:
    data_list = list(tqdm(executor.map(fetch_data, range(iterations)), total=iterations))

# 리스트 평탄화
data = [item for sublist in data_list for item in sublist]

  0%|          | 0/125 [00:00<?, ?it/s]

In [18]:
df = pd.DataFrame(data)
df

,title,link,writer,assigned,year,grad,preAbstract
0,외국인 유학생 대상국어국문학 전공 기초 교육과정 개발 연구,/search/detail/DetailView.do?p_mat_type=be54d9...,임형옥,고려대학교 대학원,2020,국내박사,The aim of this study is to develop a preparat...
1,중국 내 한국어학과 문학 교육과정의 설계와 재구성 연구 : 길림성 소재 대학의 고전...,/search/detail/DetailView.do?p_mat_type=be54d9...,정반반,공주대학교 일반대학원,2021,국내박사,이 글은 중국 길림성 소재 대학의 한국어학과 문학교육과정에 관한 내용이다. 1972...
2,인문계열 학과 법제 교육 연구,/search/detail/DetailView.do?p_mat_type=be54d9...,김태림,경희대학교 대학원,2023,국내박사,The purpose of this study is to emphasize the ...
3,대화주의에 기반한 외국인 유학생 대상 한국문학교육 연구,/search/detail/DetailView.do?p_mat_type=be54d9...,나선혜,순천대학교,2023,국내박사,The purpose of this study is to discuss litera...
4,러시아에서의 한국문학 수용 과정 연구,/search/detail/DetailView.do?p_mat_type=be54d9...,한승희,한양대학교 대학원,2021,국내박사,None
...,...,...,...,...,...,...,...
1032,일본 식민지 시기 대만 영화의 민족사실주의 연구,/search/detail/DetailView.do?p_mat_type=be54d9...,유임중,단국대학교 대학원,2022,국내박사,본 논문은 일본 식민지 시기 대만 영화의 민족사실주의에 대한 분석 연구이다. 초기 ...
1033,디지털 기호학으로의 이행에 관한 매체 철학적 연구 : 테크놀로지 문자와 컴퓨테이셔널...,/search/detail/DetailView.do?p_mat_type=be54d9...,강수환,인하대학교 대학원,2020,국내박사,To understand the signification of the digital...
1034,홍콩 지역 광동어 대중가요에 반영된 문화 요소 분석,/search/detail/DetailView.do?p_mat_type=be54d9...,종우겸,단국대학교 대학원,2024,국내박사,"본 논문은 1940년대 이후 홍콩 대중가요의 발전을 문화ㆍ역사ㆍ지리로 구 분하여, ..."
1035,北宋 詠物詩 연구,/search/detail/DetailView.do?p_mat_type=be54d9...,후미영(HOU MEILING),고려대학교 대학원,2024,국내박사,We live in a world surrounded by all things. W...


In [19]:
# csv로 저장
df.to_csv('./riss_basic.csv')

### 논문별 상세정보 수집하기

In [20]:
df['link'] = 'https://www.riss.kr' + df['link']
df['link']

,link
0,https://www.riss.kr/search/detail/DetailView.d...
1,https://www.riss.kr/search/detail/DetailView.d...
2,https://www.riss.kr/search/detail/DetailView.d...
3,https://www.riss.kr/search/detail/DetailView.d...
4,https://www.riss.kr/search/detail/DetailView.d...
...,...
1032,https://www.riss.kr/search/detail/DetailView.d...
1033,https://www.riss.kr/search/detail/DetailView.d...
1034,https://www.riss.kr/search/detail/DetailView.d...
1035,https://www.riss.kr/search/detail/DetailView.d...


In [21]:
headers_for_riss = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7,ja;q=0.6",
    "Connection": "keep-alive",
    "Cookie": "Your Cookie Here",
    "Host": "www.riss.kr",
    "Referer": None,
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "sec-ch-ua": '"Google Chrome";v="123", "Not:A-Brand";v="8", "Chromium";v="123"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "Windows"
}

In [22]:
def fetch_data(link):
    headers_for_riss["Referer"] = link
    response = request_until_success(link, headers_for_riss)
    soup = BeautifulSoup(response.text, 'lxml')

    data = {'link': link}

    title_tag = soup.find('h3', class_='title')
    data['title'] = title_tag.get_text(strip=True) if title_tag else None

    info_detail_div = soup.find('div', class_='infoDetailL')
    if info_detail_div:
        for li in info_detail_div.find_all('li'):
            key_element = li.find('span', {'class': 'strong'})
            value_element = li.find('div')
            if key_element and value_element:
                key = key_element.text.strip()
                value = value_element.text.strip()
                data[key] = value

    try:
        additional_info_div = soup.find('div', class_='content additionalInfo')
        if additional_info_div:
            text_off_divs = additional_info_div.find_all('div', class_='text off')
            title_text_dict = {}
            for div in text_off_divs:
                title = div.find_previous_sibling('p', class_='title')
                if title:
                    if title.text.strip() == "참고문헌 (Reference)":
                        title_text_dict[title.text.strip()] = [p.text.strip() for p in div.find_all('p')]
                    else:
                        title_text_dict[title.text.strip()] = div.text.strip()
            data.update(title_text_dict)
    except AttributeError:
        print(f'참고문헌 등 추가 정보 없음 : {link}')

    return data

In [24]:
import os

# CPU 코어 수 가져오기
cpu_count = os.cpu_count()

# 최대 worker 수 계산
max_workers = cpu_count * 2
max_workers

4

In [25]:
# 실행 코드
links = df['link'].tolist()  # 100개만 테스트, 다하려면 [:100] 제거
with ThreadPoolExecutor(max_workers=10) as executor:
    data_list = list(tqdm(executor.map(fetch_data, links), total=len(links)))

  0%|          | 0/1037 [00:00<?, ?it/s]

In [26]:
detail = pd.DataFrame(data_list)
detail

,link,title,저자,발행사항,학위논문사항,발행연도,작성언어,주제어,발행국(도시),기타서명,...,일반주기명,UCI식별코드,DOI식별코드,소장기관,다국어 초록 (Multilingual Abstract),목차 (Table of Contents),참고문헌 (Reference),DDC,국문 초록 (Abstract),KDC
0,https://www.riss.kr/search/detail/DetailView.d...,외국인 유학생 대상 국어국문학 전공 기초 교육과정 개발 연구,임형옥,"서울 : 고려대학교 대학원, 2020","학위논문(박사) -- 고려대학교 대학원 , 국어국문학과 , 2020. 8",2020,한국어,특정 학문 목적 한국어교육\r\n\t\t\t\t\t\t\t\r\n\t\t\t\t\t...,서울,A study on the development of a preparation cu...,...,부록수록지도교수: 김정숙참고문헌: p. 205-214,I804:11009-000000231531,10.23186/korea.000000231531.11009.0001168,고려대학교 도서관,The aim of this study is to develop a preparat...,목 차\n\n1. 서론1\n1.1. 연구의 목적 및 필요성1\n1.2. 선행연구 검...,"[1 이익섭, ""?국어학개설?"", 학연사, 2011, 2 신승용, ""「국어학 개...",NaN,NaN,NaN
1,https://www.riss.kr/search/detail/DetailView.d...,중국 내 한국어학과 문학 교육과정의 설계와 재구성 연구 : 길림성 소재 대학의 고전...,정반반\r\n\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\...,"공주 : 공주대학교 일반대학원, 2021","학위논문(박사) -- 공주대학교 일반대학원 , 국어국문학과 , 2022. 2",2021,한국어,"중국 한국어학과, 한국문학교육과정, 고전문학",충청남도,A Study on the Design and Reconstruction of Ko...,...,지도교수:권대광참고문헌 : p.164-170,I804:44004-000000031401,NaN,국립공주대학교 도서관,NaN,I. 서론 1\n 1. 연구의 필요성 및 목적 1\n 2. 선행연구 검토 5\n 3...,NaN,811 판사항(22),이 글은 중국 길림성 소재 대학의 한국어학과 문학교육과정에 관한 내용이다. 1972...,NaN
2,https://www.riss.kr/search/detail/DetailView.d...,인문계열 학과 법제 교육 연구,김태림,"서울 : 경희대학교 대학원, 2023","학위논문(박사) -- 경희대학교 대학원 , 국어국문학과 , 2023. 2",2023,한국어,"인문학, 국어국문학, 법제 교육, 저작권, 지식재산권",서울,A Study on Legal Education of Humanities Depar...,...,경희대학교 논문은 저작권에 의해 보호받습니다.지도교수: 이정재참고문헌: p. 108...,I804:11006-200000659401,NaN,경희대학교 중앙도서관,The purpose of this study is to emphasize the ...,<국문초록>ⅰ\n\nⅠ. 서론 1\n제1절 연구의 필요성 및 목적 1\n제2절 연구...,NaN,811 판사항(22),「인문계열 학과 법제 교육 연구」는 인문학 학습자들이 사회 현장에서 겪는 실제적인 ...,NaN
3,https://www.riss.kr/search/detail/DetailView.d...,대화주의에 기반한 외국인 유학생 대상 한국문학교육 연구,나선혜,"순천 : 순천대학교, 2023","학위논문(박사) -- 순천대학교 대학원 , 국어국문학과 , 2023",2023,한국어,NaN,전라남도,NaN,...,지도교수: 최현주참고문헌 수록,NaN,NaN,국립순천대학교 도서관 \r\n\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t...,The purpose of this study is to discuss litera...,Ⅰ. 서론 1\n 1. 연구의 필요성 및 목적 1\n 2. 선행 연구 검토 6\n ...,"[1 구인환, ""문학교육론"", 삼지원, 2001, 2 노상우, ""생태교육학"", ...",495.7 판사항(23),이 논문은 학문 목적 외국인 유학생을 대상으로 한 문학교육에 대해 논하였다.외국인 ...,710 판사항(6)
4,https://www.riss.kr/search/detail/DetailView.d...,러시아에서의 한국문학 수용 과정 연구 = A study on the acceptan...,한승희,"서울 : 한양대학교 대학원, 2021","학위논문(박사) -- 한양대학교 대학원 , 국어국문학과 , 2021. 2",2021,한국어,한국문학,서울,NaN,...,"권두 국문요지, 권말 Abstract 수록지도교수: 유성호참고문헌: p. 156-172",I804:11062-200000485955,NaN,국립중앙도서관 \r\n\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t...,NaN,[ 목차 ]\n\n\n국문요지\t Ⅱ\n\n제1장 서론\t 1\n제1절 연구목적과 ...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1032,https://www.riss.kr/search/detail/DetailView.d...,일본 식민지 시기 대만 영화의 민족사실주의 연구,유임중,"용인 : 단국대학교 대학원, 2022","학위논문(박사) -- 단국대학교 대학원 , 중어중문학과 중국문학전공 , 2022. 8",2022,중국어,NaN,경기도,A Study on National Realism of Taiwanese Films...,...,"단국대학교 논문은 저작권에 의해 보호받습니다.지도교수: 장호득, 임대근참고문헌 : ...",I804:11017-000000198376,NaN,단국대학교 퇴계기념도서관(중앙도서관),本論文著重對於日本殖民時期臺灣電影的民族寫實主義進行探討與研究。由於早期臺灣電影，即日殖時期...,第一章 緒論 1\n 1. 研究目的 1\n 2. 研究對象 3\n 3. 研究計劃與研究方...,NaN,NaN,본 논문은 일본 식민지 시기 대만 영화의 민족사실주의에 대한 분석 연구이다. 초기 ...,NaN
1033,https://www.riss.kr/search/detail/DetailView.d...,디지털 기호학으로의 이행에 관한 매체 철학적 연구 : 테크놀로지 문자와 컴퓨테이셔널...,강수환,"인천 : 인하대학교 대학원, 2020","학위논문(박사) -- 인하대학교 대학원 , 한국학과 한국문화콘텐츠 , 2020. 8",2020,한국어,기호학\r\n\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\r\n\t\...,인천,NaN,...,지도교수: 김만수,I804:23009-200000332560,NaN,인하대학교 도서관,To understand the signification of the digital...,Ⅰ. 서론 1\n 1. 문제 제기 및 연구 목적 1\n 2. 선행연구 검토 ...,"[1 Hegel, G. W. F., ""『법철학』"", 임석진 역, 한길사, 2008...",NaN,본 연구는 디지털 기호의 의미작용을 정확히 파악하기 위해서는 단지 생산된 기호의 표...,NaN
1034,https://www.riss.kr/search/detail/DetailView.d...,홍콩 지역 광동어 대중가요에 반영된 문화 요소 분석,종우겸,"용인 : 단국대학교 대학원, 2024","학위논문(박사) -- 단국대학교 대학원 , 중어중문학과 중국문학전공 , 2024. 2",2024,한국어,NaN,경기도,Culture Terroir of Cantonese Pop Songs in HONG...,...,"단국대학교 논문은 저작권에 의해 보호받습니다.지도교수: 배은한, 서유진참고문헌 : ...",I804:11017-000000200212,NaN,단국

In [27]:
# csv로 저장
detail.to_csv('riss_detail.csv', index=False)